In [1]:
# import numpy
import numpy as np

# import math tools
from math import pi, sqrt

# import great circle calculator
import great_circle_calculator.great_circle_calculator as gcc

# import median filter for time-series
from scipy.signal import medfilt

import matplotlib.pyplot as plt

def __Loopers__(longitude, latitude, time, f, long_min, long_max, lat_min, lat_max, threshold = 2, use_great_circle_distance = True, u_avg=0, v_avg=0):
    
    Spin = []
    # compute velocity from finite-differencing
    
    # define velocities
    velx, vely, vel = [], [], []
    accx, accy = [], []
    
    ## Compute velocities
    if use_great_circle_distance == False:
        
        # coordinates are not in longitude/latitude --> no need to compute the distances on a sphere
        
        x = longitude
        y = latitude
        
        for i in range(1, len(x)-1):
                       
            dist_x = (x[i+1]-x[i-1])*1000
            dist_y = (y[i+1]-y[i-1])*1000
        
            u = dist_x/(time[i+1]-time[i-1])
            v = dist_y/(time[i+1]-time[i-1])
        
            velx.append(u)
            vely.append(v)
            vel.append(sqrt(u**2+v**2))
    
    else:
        
        # coordinates are in longitude/latitude --> need to compute the distances on a sphere using great circle distance_calculator
        
        for i in range(1, len(longitude)-1):
        
            dist_x = np.abs(gcc.distance_between_points((longitude[i-1], latitude[i]), (longitude[i+1], latitude[i]), haversine=True))
            dist_y = np.abs(gcc.distance_between_points((longitude[i], latitude[i-1]), (longitude[i], latitude[i+1]), haversine=True))
        
            u = dist_x/(time[i+1]-time[i-1])*np.sign(longitude[i+1] - longitude[i-1])-u_avg
            v = dist_y/(time[i+1]-time[i-1])*np.sign(latitude[i+1] - latitude[i-1])-v_avg
        
            velx.append(u)
            vely.append(v)
            vel.append(sqrt(u**2+v**2))
    
    for i in range(1, len(vel)-1):
        accx.append((velx[i+1]-velx[i-1])/(time[i+1]-time[i-1]))
        accy.append((vely[i+1]-vely[i-1])/(time[i+1]-time[i-1]))
    
    #EKE = 1/2*np.mean((np.array(velx)-u_avg)**2 + np.array(vely)**2)
    
    R = []
    
    for j in range(1, len(accx)):
        EKE = 1/2*vel[j]**2
        Spin.append((velx[j]*accy[j]-vely[j]*accx[j])/EKE)
        R.append(sqrt(2*EKE)/abs(Spin[-1])/1000)
    
    days_filter = 3
    Spin = medfilt(Spin, f*days_filter+1)    
    
    idx = []
    idx.append(0)
    Loop_x, Loop_y = [], []
        
    # iterate over Spin time-series and extract segments with persistent looping
    # idx contains all zero-crossings of the Spin
    for i in range(len(Spin)-1):
        if Spin[i] >= 0 and Spin[i+1] <= 0 or Spin[i] <= 0 and Spin[i+1] >= 0:
            idx.append(i)
         
    idx.append(len(Spin)-1)
    
    sign = []
                
    if len(idx) > 0:
        for i in range(len(idx)-1):
            if idx[i+1]-idx[i] > 2:
                P = threshold*np.pi / np.median(np.abs(Spin[idx[i]:idx[i+1]]))/(3600*24)
                Persistence = (time[idx[i+1]]-time[idx[i]])/(3600*24)
                if (Persistence - threshold*P) >= 0:                
                    sign.append(np.sign(np.median(Spin[idx[i]:idx[i+1]])))
                    if long_min < longitude[idx[i+1]] < long_max and lat_min < latitude[idx[i+1]] < lat_max:
                        Loop_x.append(longitude[idx[i]:idx[i+1]+2])
                        Loop_y.append(latitude[idx[i]:idx[i+1]+2])
    
    return Loop_x, Loop_y, sign